## Seq2Seqモデル

入力文 -> LSTM Encoder -> 最終状態の隠れ層とメモリセル -> Decoder -> 生成文


### Encoder

入力文をベクトル化 <br />
LSTMの隠れ層とメモリセルの最終状態を利用

In [1]:
import numpy
import torch
from torch import nn


class LSTMEncoder(nn.Module):
    """
    LSTM Encoder
    """
    def __init__(
            self,
            vocab_size,
            embedding_size,
            rnn_input_size,
            rnn_hidden_size,
    ):
        super(LSTMEncoder, self).__init__()

        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.rnn_input_size = rnn_input_size
        self.rnn_hidden_size = rnn_hidden_size

        self.embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0) # パディングのword_idを0に指定

        self.lstm = nn.LSTM(
            rnn_input_size,
            rnn_hidden_size,
            num_layers=1, # LSTM_1(LSTM_2(...のようにLSTM を多段にすることができる  今回は一層のみ
            batch_first=True # 入力の形式を指定 (batch_size, 系列長, 分散表現の次元) のようにbatch_sizeが最初に来るよう入力
        )


    def init_hidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.rnn_hidden_size),
                torch.zeros(1, batch_size, self.rnn_hidden_size))

        
    def forward(self, input_sentence_list):
        """
        順伝搬
        :param input_sentence_list: Sentenceのリスト
        :return hidden_cell_tensors: LSTMの最終状態の(隠れ層, メモリセル)
        """ 
        self.zero_grad()
        # 系列長が長い順にソート (ミニバッチ処理の際にlstmに入力するために必要)
        input_sentence_idx_list = [i for i in range(len(input_sentence_list))]
        input_sentence_idx_list.sort(key=lambda x: len(input_sentence_list[x]), reverse=True)
        input_sentence_list = list(numpy.array(input_sentence_list)[input_sentence_idx_list])        
        length_list = [len(sentence) for sentence in input_sentence_list]

        # ミニバッチ処理の際の系列長を揃えるためのパディング処理
        pad_sentence_list = nn.utils.rnn.pad_sequence([torch.tensor(sentence.word_id_list) for sentence in input_sentence_list], batch_first=True)

        sentence_embeds = self.embeddings(pad_sentence_list) # 各単語をベクトル化
        sentence_embeds = nn.utils.rnn.pack_padded_sequence(sentence_embeds, length_list, batch_first=True)

        lstm_outputs, hidden_cell_tensors = self.lstm(sentence_embeds, self.init_hidden(len(input_sentence_list))) 
        lstm_outputs, output_lengths = nn.utils.rnn.pad_packed_sequence(lstm_outputs, batch_first=True) 

        # 元の順にソートしなおし
        idx_list = [i for i in range(len(input_sentence_list))]
        align_keys = sorted(idx_list, key=lambda x: input_sentence_idx_list[x])
        hidden_tensors, cell_tensors = hidden_cell_tensors
        hidden_tensors = hidden_tensors[:, align_keys]
        cell_tensors = cell_tensors[:, align_keys]
        hidden_cell_tensors = (hidden_tensors, cell_tensors)
        lstm_outputs = lstm_outputs[align_keys]        
        
        return hidden_cell_tensors


### Decoder

Encoderの最終状態をDecoderの初期状態として生成
#### 学習時
<img src="figures/seq2seq_train.jpg" width="520px" align="left"><br clear="all" />
<br />

#### 生成時
<img src="figures/seq2seq_gen.jpg" width="520px" align="left"><br clear="all" />
<br />


In [2]:
from data import PartOfSentence


class LSTMDecoder(nn.Module):
    """
    LSTM Decoder
    """
    def __init__(
            self,
            vocab_size,
            embedding_size,
            rnn_input_size,
            rnn_hidden_size,
    ):
        super(LSTMDecoder, self).__init__()

        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.rnn_input_size = rnn_input_size
        self.rnn_hidden_size = rnn_hidden_size
        self.output_size = vocab_size

        self.embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0) # パディングのword_idを0に指定

        self.lstm = nn.LSTM(
            rnn_input_size,
            rnn_hidden_size,
            num_layers=1,
            batch_first=True
        )
        self.output_linear = nn.Linear(rnn_hidden_size, vocab_size) # LSTMの各出力を語彙数の次元に変換する線形層
        self.loss_function = nn.CrossEntropyLoss()

        
    def forward(self, input_sentence_list, enc_hidden_cell):
        """
        順伝搬
        :param input_sentence_list: Sentenceのリスト
        :return outputs: decoder LSTM各時点での出力
        :return hidden_cell_tensors: LSTMの最終状態の(隠れ層, メモリセル) (生成時に必要)
        """ 
        self.zero_grad()
        length_list = [len(sentence) for sentence in input_sentence_list]
        
        # ミニバッチ処理の際の系列長を揃えるためのパディング処理
        pad_sentence_list = nn.utils.rnn.pad_sequence([torch.tensor(sentence.word_id_list) for sentence in input_sentence_list], batch_first=True)
        
        sentence_embeds = self.embeddings(pad_sentence_list) # 各単語をベクトル化
        sentence_embeds = nn.utils.rnn.pack_padded_sequence(sentence_embeds, length_list, batch_first=True)
        lstm_outputs, hidden_cell_tensors = self.lstm(sentence_embeds, enc_hidden_cell)
        lstm_outputs, output_lengths = nn.utils.rnn.pad_packed_sequence(lstm_outputs, batch_first=True)
        # クラス数次元に変換
        outputs = self.output_linear(lstm_outputs)

        return outputs, hidden_cell_tensors


    def forward_labels_loss(self, input_sentence_list, enc_hidden_cell):
        """
        forward()による順伝搬とloss計算
        :params input_sentence_list: Sentenceのリスト
        :return loss: 損失
        :return predicted_label_ids : 予測ラベルのid
        :return softmax_scores: 各予測の確率
        """
        # 系列長が長い順にソート (ミニバッチ処理の際にlstmに入力するために必要)
        input_sentence_idx_list = [i for i in range(len(input_sentence_list))]
        input_sentence_idx_list.sort(key=lambda x: len(input_sentence_list[x]), reverse=True)
        sorted_input_sentence_list = list(numpy.array(input_sentence_list)[input_sentence_idx_list])

        # encからの結果も合わせてソート
        hidden_tensors, cell_tensors = enc_hidden_cell
        hidden_tensors = hidden_tensors[:, input_sentence_idx_list]
        cell_tensors = cell_tensors[:, input_sentence_idx_list]
        enc_hidden_cell = (hidden_tensors, cell_tensors)

        # 順伝搬
        outputs, _ = self.forward(sorted_input_sentence_list, enc_hidden_cell)
        outputs = outputs[:, :-1, :] # <eos>入力時の出力だけ除外

        # 損失計算
        loss = 0
        for sent, output in zip(sorted_input_sentence_list, outputs):
            true_seq = torch.tensor(sent.word_id_list)[1:]
            output = output[:len(true_seq)]
            loss += self.loss_function(output, true_seq)

        # 元の順にソートしなおし
        idx_list = [i for i in range(len(input_sentence_list))]
        align_keys = sorted(idx_list, key=lambda x: input_sentence_idx_list[x])
        outputs = outputs[align_keys]
            
        # 確率値と予測ラベル
        softmax_scores = nn.functional.softmax(outputs, dim=2)
        predicted_label_ids = softmax_scores.argmax(2)

        return loss, predicted_label_ids, softmax_scores

    
    def generate(self, start_id, enc_hidden_cell, end_tag, max_generate_length):
        """
        生成（一単語ずつforward）
        :param start_id: 生成のために最初にデコーダに入力する<sos>の単語id
        :param enc_hidden_cell: エンコーダの最終状態の(h, c)
        :param end_tag: 生成を終了するための<eos>の単語id
        :param max_generate_length: 生成の最大長 (<eos>が出力されなくても生成を終了)
        :return predicted_label_id_list: 生成された単語idリスト
        """
        h = enc_hidden_cell[0]
        c = enc_hidden_cell[1]
        predicted_label_id_list = []
        word_id = start_id
        for _ in range(max_generate_length):
            outputs, (h, c) = self.forward([word_id], (h, c)) # 最初は<sos>を読み込ませて，あとは一単語ずつ生成
            softmax_scores = nn.functional.softmax(outputs, dim=2)
            predicted_label_id = softmax_scores.argmax(2)
            predicted_label_id_list.append(predicted_label_id[0].item())
            if predicted_label_id[0].item() == end_tag:
                break
            word_id = PartOfSentence(predicted_label_id[0].item()) # forwardに入力するための形式に (自作)
        return predicted_label_id_list

### Seq2Seq (Encoder+Decoder)

In [3]:
from data import PartOfSentence


class Seq2Seq(nn.Module):
    """
    Encoder-Decoderモデル
    """
    def __init__(self, 
                 vocab_size,
                 embedding_size,
                 rnn_input_size,
                 rnn_hidden_size):
        super(Seq2Seq, self).__init__()
        self.encoder = LSTMEncoder(vocab_size, embedding_size, rnn_input_size, rnn_hidden_size)
        self.decoder = LSTMDecoder(vocab_size, embedding_size, rnn_input_size, rnn_hidden_size)

        
    def forward(self, input_sentence_list):
        """
        学習の際の順伝搬とロス計算
        :param input_sentence_list: Sentenceのリスト
        :return loss: デコーダの出力から計算される損失
        :return pred_ids: 予測単語のid
        :return scores: 各予測のスコア
        """
        enc_input_sentence_list, dec_input_sentence_list = [], []
        for sent1, sent2 in input_sentence_list:
            enc_input_sentence_list.append(sent1)
            dec_input_sentence_list.append(sent2)
        # エンコーダの順伝搬
        hidden_cell_tensors = self.encoder.forward(enc_input_sentence_list)
        # デコーダの順伝搬とロス計算
        loss, pred_ids, scores = self.decoder.forward_labels_loss(dec_input_sentence_list, hidden_cell_tensors)
        return loss, pred_ids, scores

    
    def generate(self, input_sentence_list, start_tag, end_tag):
        """
        生成の際の順伝搬
        :param input_sentence_list: Sentenceのリスト
        :param start_tag: <sos>タグのタグid
        :param eos_tag: <eos>タグのタグid
        """
        pred_ids_list = []
        enc_input_sentence_list = [sent for sent, _ in input_sentence_list]
        for enc_input_sentence in enc_input_sentence_list: # 一文ずつ処理
            # 入力文のエンコード
            hidden_cell_tensors = self.encoder.forward([enc_input_sentence])
            # デコーダによる生成
            pred_ids = self.decoder.generate(PartOfSentence(start_tag), hidden_cell_tensors, end_tag, 20)
            pred_ids_list.append(pred_ids)
        return pred_ids_list
    

### 学習

In [4]:
from torch import optim

from data import Seq2SeqCorpus # 自作クラス (data.py参照)

def trainer(model, corpus):
    model.train()
    op = optim.Adam(model.parameters(), lr=0.1)
    for epoch in range(100):
        loss, pred_ids, scores = model.forward(corpus.train_sentence_pair_list)
        print(loss)
        loss.backward()
        op.step()
        model.zero_grad()


corpus = Seq2SeqCorpus("text_pairs.txt", test_file="text_pairs.txt")
seq2seq = Seq2Seq(len(corpus.word_to_id_dict), 100, 100, 100)

seq2seq.train()
op = optim.Adam(seq2seq.parameters(), lr=0.1)
for epoch in range(50):
    loss, pred_ids, scores = seq2seq.forward(corpus.train_sentence_pair_list)
    print(loss)
    loss.backward()
    op.step()
    seq2seq.zero_grad()



tensor(40.8267, grad_fn=<AddBackward0>)
tensor(21.8759, grad_fn=<AddBackward0>)
tensor(10.1230, grad_fn=<AddBackward0>)
tensor(3.7304, grad_fn=<AddBackward0>)
tensor(2.2909, grad_fn=<AddBackward0>)
tensor(1.7811, grad_fn=<AddBackward0>)
tensor(1.3168, grad_fn=<AddBackward0>)
tensor(0.8391, grad_fn=<AddBackward0>)
tensor(0.5525, grad_fn=<AddBackward0>)
tensor(0.3453, grad_fn=<AddBackward0>)
tensor(0.2708, grad_fn=<AddBackward0>)
tensor(0.2169, grad_fn=<AddBackward0>)
tensor(0.1504, grad_fn=<AddBackward0>)
tensor(0.1023, grad_fn=<AddBackward0>)
tensor(0.0689, grad_fn=<AddBackward0>)
tensor(0.0494, grad_fn=<AddBackward0>)
tensor(0.0335, grad_fn=<AddBackward0>)
tensor(0.0260, grad_fn=<AddBackward0>)
tensor(0.0206, grad_fn=<AddBackward0>)
tensor(0.0165, grad_fn=<AddBackward0>)
tensor(0.0134, grad_fn=<AddBackward0>)
tensor(0.0108, grad_fn=<AddBackward0>)
tensor(0.0087, grad_fn=<AddBackward0>)
tensor(0.0072, grad_fn=<AddBackward0>)
tensor(0.0059, grad_fn=<AddBackward0>)
tensor(0.0050, grad_fn

### 生成

In [7]:
seq2seq.eval()
start_tag_id = corpus.word_to_id_dict["<sos>"]
end_tag_id = corpus.word_to_id_dict["<eos>"]
pred_ids = seq2seq.generate(corpus.test_sentence_pair_list, start_tag_id, end_tag_id)

for (tr_sent1, tr_sent2), (sent, _), pred in zip(corpus.train_sentence_pair_list, corpus.test_sentence_pair_list, pred_ids):
    print("入力文：", tr_sent1.text)
    print("正解文：", tr_sent2.text)
    print("予測文：", "".join([corpus.id_to_word_dict[w_id] for w_id in pred]))
    print()


入力文： おはようございます。
正解文： おはよう！
予測文： おはよう！<eos>

入力文： こんにちは。
正解文： こんにちは。よろしくお願いします。
予測文： こんにちは。よろしくお願いします。<eos>

入力文： 普段何をしていますか？
正解文： 学校で働いています。
予測文： 学校で働いています。<eos>

入力文： どこに住んでいますか？
正解文： 東京に住んでいます。
予測文： 東京に住んでいます。<eos>

入力文： どこかに旅行に行こう
正解文： 北海道に行きたいです。
予測文： 北海道に行きたいです。<eos>

入力文： おいくつですか。
正解文： 二十歳です。
予測文： 二十歳です。<eos>

入力文： どんなジャンルの音楽が好きですか？
正解文： ジャズが好きです。
予測文： ジャズが好きです。<eos>

入力文： 泳ぐのは好きですか。
正解文： はい。
予測文： はい。<eos>

入力文： 誕生日はいつですか。
正解文： 11月です。
予測文： 11月です。<eos>

入力文： お会いできてよかったです。
正解文： こちらこそ。
予測文： こちらこそ。<eos>

